This file should only be used to contain the code used to download data by using the selenium 

and

Upload data to minio

In [1]:
%pip install boto3
# Libraries maybe you need to install in your environment to use selenium and chromedriver below
# %pip install selenium chromedriver-binary

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import boto3

In [4]:
# The file download processing section should be placed here
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.common.by import By
import time

driver = webdriver.Edge(service=EdgeService(EdgeChromiumDriverManager().install()))
drive_link = "https://drive.google.com/uc?export=download&id=1UJ7qWyW9vJIac05W8ynbTZE8ImuRkTVK"
driver.get(drive_link)

time.sleep(10)

driver.quit()

from dotenv import load_dotenv
from zipfile import ZipFile

load_dotenv()
DATA_FILE_PATH = os.getenv('DEFAULT_DOWNLOAD_DIRECTORY') + "\\data.zip"
with ZipFile(file=DATA_FILE_PATH, mode="r") as zipfile:
    zipfile.extractall(members=["data.json", "data2.json", "data3.json"])
#...........................................................

In [ ]:
# Define S3 storage

obj_storage_access_key = os.getenv('OBJ_STORAGE_ACCESS_KEY', 'demo-access-key')
obj_storage_secret_key = os.getenv('OBJ_STORAGE_SECRET_KEY', 'demo-secret-key')
obj_storage_endpoint = os.getenv('OBJ_STORAGE_ENDPOINT', 'http://localhost:9000')


In [ ]:
# Function to upload file to minio
def upload_file_to_minio(file_path, minio_bucket, minio_object_name):
    s3c = boto3.resource('s3',
                        endpoint_url=obj_storage_endpoint,
                        aws_access_key_id=obj_storage_access_key,
                        aws_secret_access_key=obj_storage_secret_key,
                        config=boto3.session.Config(signature_version='s3v4'),
                        verify=False
                        )
    s3c.Bucket(minio_bucket).upload_file(file_path, minio_object_name)


In [ ]:
# Upload file to minio
upload_file_to_minio('data.json', 'data', 'data-raw/data.json')
upload_file_to_minio('data2.json', 'data', 'data-raw/data2.json')
upload_file_to_minio('data3.json', 'data', 'data-raw/data3.json')